#####################CibersortX网页操作: https://cibersortx.stanford.edu/

#####################详见PPT

### Step4. 可视化

输出结果一般多用于做比较多组间的细胞占比差异或生存分析

In [136]:
celltype <- "Epithelial"  ###修改细胞类型的名称

#### 6.1 比较多组间的细胞占比差异

In [137]:
library(survival)  
library(survminer) # 用来做km曲线
library(RColorBrewer)
library(corrplot)

if(T){
  mytheme <- theme(plot.title = element_text(size = 12,color="black",hjust = 0.5),
                   axis.title = element_text(size = 12,color ="black"), 
                   axis.text = element_text(size= 12,color = "black"),
                   panel.grid.minor.y = element_blank(),
                   panel.grid.minor.x = element_blank(),
                   axis.text.x = element_text(angle = 45, hjust = 1 ),
                   panel.grid=element_blank(),
                   legend.position = "top",
                   legend.text = element_text(size= 12),
                   legend.title= element_text(size= 12)
  )}

In [138]:
TME_data <- read.csv(paste0("CIBERSORTx_", celltype, ".csv"),header = T,row.names = 1)
head(TME_data)

,Epithelial_c8,Epithelial_c3,Epithelial_c0,Epithelial_c5,Epithelial_c2,Epithelial_c1,Epithelial_c7,Epithelial_c9,Epithelial_c6,Epithelial_c4,Epithelial_c10,P.value,Correlation,RMSE
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
GSE193876_CTRL_1,0.000000e+00,0,0.012252797,0.0000000,0.4123853,0.0000000,0.08753058,0.033408393,0.3795191,0.07490390,0.000000e+00,0,0.9980951,0.12699095
GSE193876_CTRL_2,0.000000e+00,0,0.021020748,0.0000000,0.4155788,0.0000000,0.08238558,0.027283893,0.3935502,0.06018082,0.000000e+00,0,0.9982839,0.13831088
GSE193876_CTRL_3,0.000000e+00,0,0.019043737,0.0000000,0.3454307,0.0000000,0.11545081,0.026270955,0.4049268,0.08887699,0.000000e+00,0,0.9973660,0.08383477
GSE193876_CTRL_4,0.000000e+00,0,0.042129907,0.0255645,0.5451255,0.0000000,0.04118360,0.002560516,0.2777931,0.06564282,0.000000e+00,0,0.9992588,0.48531148
GSE193876_LPS_48h_1,2.236333e-06,0,0.007407694,0.0000000,0.2890434,0.2496191,0.06105783,0.018128523,0.2031955,0.17061533,9.303219e-04,0,0.9958280,0.11690644
GSE193876_LPS_48h_2,1.611906e-03,0,0.000000000,0.0000000,0.3389210,0.1524483,0.06410591,0.015637000,0.2504791,0.17676446,3.232919e-05,0,0.9973757,0.07300527


In [139]:
# 6.1.1 数据粗处理

library(dplyr)

# 排除最后三列
TME_data_sub <- TME_data[, 1:(ncol(TME_data)-3)]

TME_New = TME_data_sub
TME_New$sample <- row.names(TME_New)
head(TME_New)


# 筛选特定行（在bulk和scRNA数据保存出来的时候，要统一好样本名称）
library(dplyr)
library(stringr)

TME_New <- TME_New %>%
  mutate(OS = case_when(
    str_detect(sample, "CTRL") ~ 0,
    str_detect(sample, "LPS")  ~ 1,
    TRUE ~ NA_real_  # 其他情况填 NA
  ))



,Epithelial_c8,Epithelial_c3,Epithelial_c0,Epithelial_c5,Epithelial_c2,Epithelial_c1,Epithelial_c7,Epithelial_c9,Epithelial_c6,Epithelial_c4,Epithelial_c10,sample
,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
GSE193876_CTRL_1,0.000000e+00,0,0.012252797,0.0000000,0.4123853,0.0000000,0.08753058,0.033408393,0.3795191,0.07490390,0.000000e+00,GSE193876_CTRL_1
GSE193876_CTRL_2,0.000000e+00,0,0.021020748,0.0000000,0.4155788,0.0000000,0.08238558,0.027283893,0.3935502,0.06018082,0.000000e+00,GSE193876_CTRL_2
GSE193876_CTRL_3,0.000000e+00,0,0.019043737,0.0000000,0.3454307,0.0000000,0.11545081,0.026270955,0.4049268,0.08887699,0.000000e+00,GSE193876_CTRL_3
GSE193876_CTRL_4,0.000000e+00,0,0.042129907,0.0255645,0.5451255,0.0000000,0.04118360,0.002560516,0.2777931,0.06564282,0.000000e+00,GSE193876_CTRL_4
GSE193876_LPS_48h_1,2.236333e-06,0,0.007407694,0.0000000,0.2890434,0.2496191,0.06105783,0.018128523,0.2031955,0.17061533,9.303219e-04,GSE193876_LPS_48h_1
GSE193876_LPS_48h_2,1.611906e-03,0,0.000000000,0.0000000,0.3389210,0.1524483,0.06410591,0.015637000,0.2504791,0.17676446,3.232919e-05,GSE193876_LPS_48h_2


In [140]:
# 6.1.2 融合数据
#reshape2::melt() 函数默认不会自动将 OS 和 sample 作为 id.vars，除非它们已经显式指定为标识符列。默认情况下，melt() 只会将 数据框中的列 转换为长格式，其中 所有列都会被认为是变量列，除非你显式地指定了 id.vars。
# 这里使用的reshape2::melt是被改造之后的，（不知道来自哪个包）默认标识符列包括OS，Sample
TME_filtered = reshape2::melt(TME_New,id.vars=c("OS","sample"))
colnames(TME_filtered)=c("Group","Sample","Celltype","Composition") #设置行名
head(TME_filtered)


,Group,Sample,Celltype,Composition
,<dbl>,<chr>,<fct>,<dbl>
1,0,GSE193876_CTRL_1,Epithelial_c8,0.000000e+00
2,0,GSE193876_CTRL_2,Epithelial_c8,0.000000e+00
3,0,GSE193876_CTRL_3,Epithelial_c8,0.000000e+00
4,0,GSE193876_CTRL_4,Epithelial_c8,0.000000e+00
5,1,GSE193876_LPS_48h_1,Epithelial_c8,2.236333e-06
6,1,GSE193876_LPS_48h_2,Epithelial_c8,1.611906e-03


### ————————之选bulk数据中的一部分——————

In [141]:
### 只选一部分
TME_filtered <- TME_filtered[!grepl("GSE247266", TME_filtered$Sample), ]

plot_order = TME_filtered %>% 
  group_by(Celltype) %>% 
  summarise(m = median(Composition)) %>% 
  arrange(desc(m)) %>% 
  pull(Celltype)
plot_order

TME_filtered$Celltype = factor(TME_filtered$Celltype,levels = plot_order)

TME_filtered$Group <- factor(TME_filtered$Group)

[1] Epithelial_c2  Epithelial_c6  Epithelial_c4  Epithelial_c7  Epithelial_c1 
 [6] Epithelial_c9  Epithelial_c0  Epithelial_c8  Epithelial_c3  Epithelial_c5 
[11] Epithelial_c10
11 Levels: Epithelial_c8 Epithelial_c3 Epithelial_c0 ... Epithelial_c10

In [142]:
library(ggplot2)
library(ggpubr)
# options(repr.plot.width = 18, repr.plot.height = 10)

pdf(paste0("bulk1_geom_boxplot_", celltype, ".pdf"), height = 10,width = 18) #单位是英寸
ggplot(TME_filtered, aes(x = Celltype, y = Composition)) + 
  geom_boxplot(aes(fill = Group),
               position = position_dodge(0.5),
               width = 0.5,
               outlier.alpha = 0) + 
  scale_fill_manual(values = c("#1CB4B8", "#EB7369")) +
  theme_classic() +
  theme(
    text = element_text(size = 16),
    axis.text = element_text(size = 16),
    axis.title = element_text(size = 16),
    legend.text = element_text(size = 16),
    legend.title = element_text(size = 16),
    plot.title = element_text(size = 16, face="bold", hjust = 0.5)
  ) +
  stat_compare_means(
    aes(group = Group),
    label = "p.signif",
    method = "wilcox.test",
    hide.ns = FALSE,
    size = 6   # <-- 调整显著性标记的字体大小
  )
dev.off()
# 保存图片
# ggsave("Step1.CibersortX_group.pdf", height = 10, width = 18, units = "cm")


agg_record_1757523092 
                    2

In [ ]:
sss

### ————————之选bulk数据中的一部分——————

In [ ]:
plot_order = TME_filtered %>% 
  group_by(Celltype) %>% 
  summarise(m = median(Composition)) %>% 
  arrange(desc(m)) %>% 
  pull(Celltype)
plot_order

TME_filtered$Celltype = factor(TME_filtered$Celltype,levels = plot_order)

TME_filtered$Group <- factor(TME_filtered$Group)

In [116]:
library(ggplot2)
library(ggpubr)
# options(repr.plot.width = 18, repr.plot.height = 10)

pdf(paste0("geom_boxplot_", celltype, ".pdf"), height = 10,width = 18) #单位是英寸
ggplot(TME_filtered, aes(x = Celltype, y = Composition)) + 
  geom_boxplot(aes(fill = Group),
               position = position_dodge(0.5),
               width = 0.5,
               outlier.alpha = 0) + 
  scale_fill_manual(values = c("#1CB4B8", "#EB7369")) +
  theme_classic() +
  theme(
    text = element_text(size = 16),
    axis.text = element_text(size = 16),
    axis.title = element_text(size = 16),
    legend.text = element_text(size = 16),
    legend.title = element_text(size = 16),
    plot.title = element_text(size = 16, face="bold", hjust = 0.5)
  ) +
  stat_compare_means(
    aes(group = Group),
    label = "p.signif",
    method = "wilcox.test",
    hide.ns = FALSE,
    size = 6   # <-- 调整显著性标记的字体大小
  )
dev.off()
# 保存图片
# ggsave("Step1.CibersortX_group.pdf", height = 10, width = 18, units = "cm")


agg_record_1824533346 
                    2

#### 6.2 生存分析(没有time不分析)

#### 6.3 Bulk反卷积结果总览(需要根据geom_boxplot选显著的细胞亚型)

In [107]:
sig_cell = "_c1"

In [91]:
# 这里需要指定Celltype
mypalette <- colorRampPalette(brewer.pal(8,"Set1"))
plot_order = filter(TME_filtered, Celltype == paste0(celltype,sig_cell)) %>% 
  arrange(desc(Composition)) %>% 
  pull(Sample)
head(plot_order)

[1] GSE193876_LPS_48h_1 GSE193876_LPS_48h_4 GSE193876_LPS_48h_2
[4] GSE193876_LPS_48h_3 GSE193876_CTRL_1    GSE193876_CTRL_2   
14 Levels: GSE247266_CTRL_2 GSE193876_CTRL_4 ... GSE247266_LPS_2

In [92]:
pdf(paste0("geom_bar_", celltype,sig_cell, ".pdf"), height = 10,width = 12) #单位是英寸

# 将 Sample 按 plot_order 排序
TME_filtered$Sample <- factor(TME_filtered$Sample, levels = plot_order)

ggplot(TME_filtered, aes(Sample, Composition, fill = Celltype)) + 
  geom_bar(stat = "identity") +
  labs(fill = "Cell Type", x = "", y = "Estimated Proportion") + 
  theme_bw() +
  theme(
    axis.text.x = element_text(angle = 45, hjust = 1, size = 10),  # 显示 x 轴标签并旋转45°
    axis.ticks.x = element_line(),                                   # 显示刻度线
    legend.position = "bottom"
  ) + 
  scale_y_continuous(expand = c(0.01, 0)) +
  scale_fill_manual(values = mypalette(22))

dev.off()


agg_record_303496278 
                   2

## 6.4 相关性分析-可选可选（散点图每个点表示一个样本）
TME_data_sub为所有样本，用的融合前的数据

In [93]:
library(psych)

In [94]:
# 选数值列
library(dplyr)


numeric_data <- TME_data_sub %>% dplyr::select(where(is.numeric))


numeric_data <- numeric_data %>%
  select(where(~ any(. != 0)))


# 计算 Spearman 相关
library(psych)
corda.cell.type <- corr.test(numeric_data, numeric_data, method = "sp")

# 转成矩阵
r.df <- as.matrix(corda.cell.type$r)
p.df <- as.matrix(corda.cell.type$p.adj)


In [95]:
# corda.cell.type <- psych::corr.test(TME_filtered, TME_filtered, method = "sp")
# r.df = as.matrix(corda.cell.type$r)
# p.df = as.matrix(corda.cell.type$p.adj)

In [96]:
options(repr.plot.width = 10, repr.plot.height = 10)

pdf(paste0("corrplot_", celltype, ".pdf"),height = 10,width = 10) #单位是英寸

corrplot(as.matrix(r.df),
         method = "color",
         col=rev(COL2("RdBu", 200)),
         type = c("upper"),
         p.mat = p.df,#是否添加了P值，如果是NULL，则sig.level到pch.cex参数无效
         sig.level = 0.05,#P值检验水平设置
         insig = c("label_sig"),
         pch.cex = 1, 
         tl.col = "black",
         tl.cex = 0.8,#文本标签的大小
         na.label = "",#NA值如何表示
         na.label.col = "#bcb9b5",#表示NA值符号的颜色
)
dev.off()

agg_record_78443232 
                  2

### 选择上一步有相关性的细胞亚型

In [97]:
options(repr.plot.width = 10, repr.plot.height = 10)

pdf(paste0("geom_point_", celltype, ".pdf"),height = 10,width = 10) #单位是英寸

ggplot(data = TME_data_sub, aes(x = Epithelial_c1, y = Epithelial_c4)) + 
  geom_point(size=1,show.legend=T)+ 
  labs(title = "Epithelial_c1 vs. Epithelial_c4", x="Epithelial_c1 (%)",y="Epithelial_c4 (%)") + 
  geom_smooth(method="lm",formula= 'y ~ x',color="red",linetype=2,)+theme_bw()+ 
  stat_cor(method = "sp",
           label.sep = "\n",
           label.y.npc = "top",
           size = 2.8) +
  mytheme

dev.off()

agg_record_2125625350 
                    2

In [40]:
head(TME_data_sub)

,Endothelial_c0,Endothelial_c2,Endothelial_c6,Endothelial_c9,Endothelial_c3,Endothelial_c7,Endothelial_c4,Endothelial_c8,Endothelial_c1,Endothelial_c5
,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<int>
GSE193876_CTRL_1,0.16508557,0.2031086,0.028903413,0.00000000,0,0.023825665,0.1201018,0.4589750,0,0
GSE193876_CTRL_2,0.06703899,0.1298213,0.043214408,0.00000000,0,0.021758063,0.2096952,0.5284720,0,0
GSE193876_CTRL_3,0.09123348,0.1744604,0.037153710,0.00000000,0,0.021783053,0.1572604,0.5181090,0,0
GSE193876_CTRL_4,0.06427141,0.1595240,0.102830970,0.00000000,0,0.023722325,0.1104123,0.5392390,0,0
GSE193876_LPS_48h_1,0.08979399,0.1464978,0.009746823,0.00000000,0,0.007954339,0.3746053,0.3714017,0,0
GSE193876_LPS_48h_2,0.12056269,0.1048283,0.012534364,0.00611056,0,0.010712608,0.3981697,0.3470817,0,0
